## Notebook de test
Test du **notebook**  
*Texte en italique*

In [13]:
import pandas as pd

In [14]:
#Exemple de jointure

# DataFrame 1 : Clients
df_clients = pd.DataFrame({
    'client_id': [1, 2, 3, 4],
    'nom': ['Alice', 'Bob', 'Charlie', 'David'],
    'ville': ['Paris', 'Lyon', 'Marseille', 'Toulouse']
})

# DataFrame 2 : Commandes
df_commandes = pd.DataFrame({
    'commande_id': [101, 102, 103, 104],
    'client_id': [1, 2, 3, 1],
    'montant': [250, 150, 200, 300]
})

display(df_clients)

display(df_commandes)

,client_id,nom,ville
0,1,Alice,Paris
1,2,Bob,Lyon
2,3,Charlie,Marseille
3,4,David,Toulouse


,commande_id,client_id,montant
0,101,1,250
1,102,2,150
2,103,3,200
3,104,1,300


In [15]:
# Effectuer une jointure (INNER JOIN) sur la colonne 'client_id'
df_jointure = df_clients.merge(df_commandes, on='client_id')
df_jointure

,client_id,nom,ville,commande_id,montant
0,1,Alice,Paris,101,250
1,1,Alice,Paris,104,300
2,2,Bob,Lyon,102,150
3,3,Charlie,Marseille,103,200


In [16]:
# Effectuer une jointure (LEFT JOIN)
df_jointure2 = df_clients.merge(df_commandes, on='client_id', how='left')
df_jointure2

,client_id,nom,ville,commande_id,montant
0,1,Alice,Paris,101.0,250.0
1,1,Alice,Paris,104.0,300.0
2,2,Bob,Lyon,102.0,150.0
3,3,Charlie,Marseille,103.0,200.0
4,4,David,Toulouse,NaN,NaN


In [17]:
#Exemple de group by

# Grouper par 'client_id' et calculer le montant total dépensé
df_groupby = df_jointure.groupby('client_id')['montant'].sum().reset_index()

# Afficher le DataFrame résultant
display(df_groupby)

,client_id,montant
0,1,550
1,2,150
2,3,200


In [60]:
pd.DataFrame(df_jointure.groupby('client_id')['montant'].sum())

,montant
client_id,
1,550
2,150
3,200


In [61]:
df_jointure.groupby(['client_id']).agg(somme_total=("montant", "sum"))

,somme_total
client_id,
1,550
2,150
3,200


In [19]:
# Grouper par 'client_id' et calculer le montant total dépensé
df_groupby2 = df_jointure.groupby(['client_id']).agg(somme_total=("montant", "sum")).reset_index()

# Afficher le DataFrame résultant
display(df_groupby2)

,client_id,somme_total
0,1,550
1,2,150
2,3,200


### Repérer des données se trouvant dans un dataframe mais pas dans l'autre
Utilisation de la fonction merge avec un merge 'left' et indicator=True pour repérer uniquement les données présentes dans le premier dataframe

In [44]:
import pandas as pd

# df1 : Liste des personnes dans df1
df1 = pd.DataFrame({
    'nom': ['Dupont', 'Martin', 'Durand', 'Bernard'],
    'prenom': ['Alice', 'Bob', 'Charlie', 'David']
})

# df2 : Liste des personnes dans df2
df2 = pd.DataFrame({
    'nom': ['Martin', 'Bernard', 'Lefevre'],
    'prenom': ['Bob', 'David', 'Emma']
})

print("DataFrame df1:")
display(df1)
print("\nDataFrame df2:")
display(df2)

DataFrame df1:


,nom,prenom
0,Dupont,Alice
1,Martin,Bob
2,Durand,Charlie
3,Bernard,David



DataFrame df2:


,nom,prenom
0,Martin,Bob
1,Bernard,David
2,Lefevre,Emma


In [45]:
# Effectuer une jointure externe gauche (left join)
# indicator=True : Ajoute une colonne _merge indiquant l'origine de chaque ligne.
df_diff = pd.merge(df1, df2, on=['nom', 'prenom'], how='left', indicator=True)
df_diff

,nom,prenom,_merge
0,Dupont,Alice,left_only
1,Martin,Bob,both
2,Durand,Charlie,left_only
3,Bernard,David,both


In [46]:
# Sélectionner uniquement les lignes où '_merge' vaut 'left_only'
df_diff2 = df_diff[df_diff['_merge'] == 'left_only']
df_diff2

,nom,prenom,_merge
0,Dupont,Alice,left_only
2,Durand,Charlie,left_only


In [49]:
print("\nPersonnes présentes dans df1 mais pas dans df2:")
df_diff2[['nom','prenom']]


Personnes présentes dans df1 mais pas dans df2:


,nom,prenom
0,Dupont,Alice
2,Durand,Charlie


In [50]:
df2[['nom', 'prenom']].to_dict(orient='list')

{'nom': ['Martin', 'Bernard', 'Lefevre'], 'prenom': ['Bob', 'David', 'Emma']}

In [41]:
# Autre méthode (utilisation de la fonction isin)
df1[['nom', 'prenom']].isin(df2[['nom', 'prenom']].to_dict(orient='list'))

,nom,prenom
0,False,False
1,True,True
2,False,False
3,True,True


In [52]:
~df1[['nom', 'prenom']].isin(df2[['nom', 'prenom']].to_dict(orient='list')).all(axis=1)

0     True
1    False
2     True
3    False
dtype: bool

In [54]:
mask = ~df1[['nom', 'prenom']].isin(df2[['nom', 'prenom']].to_dict(orient='list')).all(axis=1)
mask

0     True
1    False
2     True
3    False
dtype: bool

In [55]:
df1[mask]

,nom,prenom
0,Dupont,Alice
2,Durand,Charlie
